# Kimberlite TFA Inversion

This notebook performs the inversion using Levenberg-Marquadt's algorithm of total field anomaly (TFA) data on a regular grid of a synthetic kimberlitic model.

In [23]:
import numpy as np
import matplotlib.pyplot as plt
import cPickle as pickle
import os

from fatiando import utils
from fatiando.gravmag import polyprism
from fatiando.mesher import PolygonalPrism
from fatiando.vis import mpl, myv
from matplotlib import colors, ticker, cm
from IPython.display import Image as img
from matplotlib.mlab import normpdf

### Auxiliary functions

In [24]:
import sys
sys.path.insert(0, '../../code')

import mag_polyprism_functions as mfun

# Input

### Importing model parameters

In [25]:
model_dir = 'data/model.pickle'
grid_dir = '../grids/flightlines_grid_2100pts_100km2_-150m.pickle'
data_dir = 'data/flightlines_grid_2100pts_100k/data.pickle'

In [26]:
with open(model_dir) as w:
        model = pickle.load(w)

### Observation points and observed data

In [27]:
with open(grid_dir) as w:
        grid = pickle.load(w)

In [28]:
with open(data_dir) as w:
        data = pickle.load(w)

In [29]:
# observed data and observation points
dobs = data['amf_obs']
xp = grid['x']
yp = grid['y']
zp = grid['z']
N = grid['N']

### Parameters of the initial model

In [30]:
M = 20 # number of vertices per prism
L = 5 # number of prisms
P = L*(M+2) + 1 # number of parameters

# magnetization direction
incs = model['inc']# - 5.
decs = model['dec']# - 5.

# depth to the top, thickness and radius
z0 = model['z0']# - 50.
dz = 450.
r = 2000.

# total magnetization
intensity = model['intensity']# + 1.
props = {'magnetization': utils.ang2vec(
        intensity, incs, decs)}
rin = np.zeros(M) + r
m0 = np.hstack((rin, np.zeros(2)))
m0 = np.resize(m0, P - 1) 
m0 = np.hstack((m0, dz)) # inicial parameters vector
model0 = mfun.param2polyprism(m0, M, L, z0, props) # list of classes of prisms

# main field
inc, dec = data['main_field']

# predict data
d0 = np.sqrt(polyprism.bx(xp, yp, zp, model0)**2. +
             polyprism.by(xp, yp, zp, model0)**2. +
             polyprism.bz(xp, yp, zp, model0)**2.)

In [31]:
xp.size

2100

### Limits

In [32]:
# limits for parameters in meters
rmin = 10.
rmax = 3000.
x0min = -4000.
x0max = 4000.
y0min = -4000.
y0max = 4000.
dzmin = 10.
dzmax = 1000.

mmin, mmax = mfun.build_range_param(M, L, rmin, rmax, x0min, x0max, y0min, y0max, dzmin, dzmax)

### Variation

In [33]:
# variation for derivatives
deltax = 0.01*np.max(1000)
deltay = 0.01*np.max(1000)
deltar = 0.01*np.max(1000)
deltaz = 0.01*np.max(1000)

### Outcropping parameters

In [34]:
# outcropping body parameters
m_out = np.zeros(M + 2)
#m_out = model['param_vec'][:M+2]

### Folder to save the results

In [35]:
if grid_dir[9:20] == 'flightlines':
    if incs == model['inc'] and decs == model['dec']:
        mypath = data_dir[:-11]+'amf_inversion/inc%d_dec%d_r%d_int%d_z0%d_dz%d' % (incs, decs, rin[0], np.linalg.norm(props['magnetization']), z0, dz)
        if not os.path.isdir(mypath):
           os.makedirs(mypath)
    else:
        mypath = data_dir[:-11]+'amf_inversion/inc%d_dec%d_r%d_int%d_z0%d_dz%d' % (incs, decs, rin[0], np.linalg.norm(props['magnetization']), z0, dz)
        if not os.path.isdir(mypath):
           os.makedirs(mypath)
else:
    if incs == model['inc'] and decs == model['dec']:
        mypath = data_dir[:-11]+'amf_inversion/inc%d_dec%d_r%d_int%d_z0%d_dz%d' % (incs, decs, rin[0], np.linalg.norm(props['magnetization']), z0, dz)
        if not os.path.isdir(mypath):
           os.makedirs(mypath)
    else:
        mypath = data_dir[:-11]+'amf_inversion/inc%d_dec%d_r%d_int%d_z0%d_dz%d' % (incs, decs, rin[0], np.linalg.norm(props['magnetization']), z0, dz)
        if not os.path.isdir(mypath):
           os.makedirs(mypath)

In [36]:
# output of inversion
inversion = dict()

### Regularization parameters

In [37]:
#lamb = th*0.01 # Marquadt's parameter
lamb = 10.0
dlamb = 10.      # step for Marquadt's parameter

a1 = 1.0e-3  # adjacent radial distances within each prism
a2 = 1.0e-4   # vertically adjacent radial distances
a3 = 0.     # outcropping cross-section
a4 = 0.     # outcropping origin
a5 = 1.0e-4     # vertically adjacent origins
a6 = 1.0e-6   # zero order Tikhonov on adjacent radial distances
a7 = 1.0e-5     # zero order Tikhonov on thickness of each prism

In [38]:
delta = np.array([deltax, deltay, deltar, deltaz])
alpha = np.array([a1, a2, a3, a4, a5, a6, a7])

In [39]:
itmax = 30
itmax_marq = 10
tol = 1.0e-4     # stop criterion

In [40]:
inversion['x'] = xp
inversion['y'] = yp
inversion['z'] = zp
inversion['observed_data'] = dobs

In [41]:
inversion['inc_dec'] = [incs, decs]
inversion['z0'] = z0
inversion['initial_dz'] = dz
inversion['intial_r'] = r
inversion['initial_estimate'] = model0
inversion['initial_data'] = d0
inversion['limits'] = [rmin, rmax, x0min, x0max, y0min, y0max, dzmin, dzmax]
inversion['regularization'] = alpha
inversion['tol'] = tol
inversion['main_field'] = [data['main_field'][0], data['main_field'][1]]

### Inversion

In [42]:
d_fit, m_est, model_est, phi_list, model_list, res_list = mfun.levmarq_amf(
    xp, yp, zp, m0, M, L, delta,
    itmax, itmax_marq, lamb,
    dlamb, tol, mmin, mmax,
    m_out, dobs, props, alpha, z0, dz
)

it:  0   it_marq:  0   lambda: 1e+01   init obj.: 3.58742e+05  fin obj.: 1.63263e+05
it:  1   it_marq:  0   lambda: 1e+00   init obj.: 1.63263e+05  fin obj.: 5.04609e+04
it:  2   it_marq:  0   lambda: 1e-01   init obj.: 5.04609e+04  fin obj.: 2.74362e+03
it:  3   it_marq:  0   lambda: 1e-02   init obj.: 2.74362e+03  fin obj.: 1.18560e+04
it:  3   it_marq:  1   lambda: 1e-01   init obj.: 2.74362e+03  fin obj.: 9.62145e+02
it:  4   it_marq:  0   lambda: 1e-02   init obj.: 9.62145e+02  fin obj.: 4.74843e+03
it:  4   it_marq:  1   lambda: 1e-01   init obj.: 9.62145e+02  fin obj.: 7.09304e+02
it:  5   it_marq:  0   lambda: 1e-02   init obj.: 7.09304e+02  fin obj.: 1.46288e+03
it:  5   it_marq:  1   lambda: 1e-01   init obj.: 7.09304e+02  fin obj.: 3.99659e+02
it:  6   it_marq:  0   lambda: 1e-02   init obj.: 3.99659e+02  fin obj.: 7.26955e+02
it:  6   it_marq:  1   lambda: 1e-01   init obj.: 3.99659e+02  fin obj.: 2.81253e+02
it:  7   it_marq:  0   lambda: 1e-02   init obj.: 2.81253e+02  fi

# Results

In [43]:
inversion['data_fit'] = d_fit
inversion['estimate'] = m_est
inversion['prisms'] = model_est
inversion['estimated_models'] = model_list
inversion['objective'] = phi_list
inversion['residual'] = dobs - d_fit
inversion['residual_list'] = res_list

In [44]:
file_name = mypath+'/inversion.pickle'
with open(file_name, 'w') as f:
    pickle.dump(inversion, f)